In [ ]:
import pandas as pd
import json
import requests, time, datetime

In [ ]:
user_lst = open(r'steam_user_id.txt')
user_lst = user_lst.readlines()[::2]

## Get User Data

In [ ]:
start_time = time.time()

api_key = '0930256E4596FEB25C73092F55A028FC'
url = 'https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001'
current_count = 0
path = 'data/user_data.txt'

with open(path, 'w') as f:
    for user_id in user_lst:
        params = {
            'key' : api_key,
            'steamid' : user_id.strip(),
            'include_appinfo' : 0, 
            'include_played_free_games' : 1,
            'format' : 'json'
        }
        for i in range(3):
            try: 
                r = requests.get(url, params)
                result = r.json().get('response').get('games')
                dict_temp = {}
                dict_temp[user_id.strip()] = result
                current_count = current_count+1
                break
            except Exception as e:
                print(e)
                time.sleep(5)
        f.write(json.dumps(dict_temp))
        f.write('\n')
        if current_count > 0 and current_count % 200 == 0:
            print(current_count, datetime.now)
            time.sleep(300)
        else:
            time.sleep(.5)
            
print('Completed')
print('total time: ', start_time-time.time())

## Get List of All Games

In [ ]:
url = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'
r = requests.get(url)
dic_app_list = r.json()
lst_app_id = [i.get('appid') for i in dic_app_list.get('applist').get('apps')]
len(lst_app_id)

In [ ]:
with open('app_id.json', 'w') as f:
    json.dump(lst_app_id, f)

## Get Game Info

In [ ]:
start_time = time.time()

current_count = 0
path = 'data/app_detail.txt'

with open(path, 'w') as f:
    for app_id in lst_app_id:
        url = ('http://store.steampowered.com/api/appdetails?appids=%s') % (app_id)
        for i in range(3):
            try:
                r = requests.get(url)
                result = r.json()
                current_count = current_count+1
                break
            except Exception as e:
                print(e)
                time.sleep(5)
        f.write(json.dumps(result))
        f.write('\n')
        if current_count > 0 and current_count % 200 == 0:
            print(current_count, datetime.now)
            time.sleep(300)
        else:
            time.sleep(.5)
            
print('Completed')
print('total time: ', start_time-time.time())